<a href="https://colab.research.google.com/github/jayw20230711/COLAB/blob/main/Insurance_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %% [markdown]
"""
# Insurance Customer Service Chatbot (RAG System)
Build a knowledge-grounded chatbot for insurance queries using Retrieval-Augmented Generation*
"""
# %%
!pip install -q langchain openai faiss-cpu pypdf sentence-transformers streamlit tiktoken
# %%
from IPython.display import HTML, display
import sys

def set_up_colab():
    display(HTML('''
    <script>
      function startStreamlit() {
        var button = document.querySelector("button");
        button.disabled = true;
        button.textContent = "Running Streamlit...";

        google.colab.kernel.proxyPort(8501, {
          'cache': false,
          'onBackendReady': function() {
            var iframe = document.createElement('iframe');
            iframe.src = 'https://localhost:8501';
            iframe.width = '100%';
            iframe.height = '600px';
            iframe.frameBorder = 0;

            var output = document.getElementById('output');
            output.innerHTML = '';
            output.appendChild(iframe);
          }
        });
      }
    </script>
    <button onclick="startStreamlit()">Launch Chatbot UI</button>
    <div id="output"></div>
    '''))
# %% [markdown]

In [ ]:
"""
## Step 1: Set Up Knowledge Base
We'll use sample insurance documents (you should replace with your actual data)
"""
# %%
!mkdir -p sample_data
!echo "Home Insurance Policy\nCoverage includes:\n- Fire damage\n- Theft\n- Natural disasters (excluding floods)\nDeductible: $500" > sample_data/home_policy.txt
!echo "Claims Process:\n1. Report incident within 72 hours\n2. Submit claim form\n3. Provide evidence (photos, police report)\n4. Wait 5-7 business days for processing" > sample_data/claims_process.txt
# %%
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = DirectoryLoader('sample_data/', glob="**/*.txt", loader_cls=TextLoader)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    length_function=len
)
chunks = text_splitter.split_documents(documents)

print(f"Split {len(documents)} documents into {len(chunks)} chunks")
print("Sample chunk:", chunks[0].page_content[:100] + "...")
# %% [markdown]


Split 2 documents into 2 chunks
Sample chunk: Claims Process:\n1. Report incident within 72 hours\n2. Submit claim form\n3. Provide evidence (phot...


In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain_openai import OpenAIEmbeddings

In [ ]:
!pip install -q langchain-openai

In [ ]:
"""
## Step 2: Create Vector Database
We'll use FAISS for efficient similarity search
"""
# %%
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import FAISS
import os

# Get your OpenAI API key: https://platform.openai.com/api-keys
os.environ['OPENAI_API_KEY'] = 'bullshit-0QiWnyC1YmMdpgs8A'  # Replace with your actual key

# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_db = FAISS.from_documents(chunks, embeddings)
vector_db.save_local("insurance_vector_db")
# %% [markdown]

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
pip install -U langchain-community

In [ ]:
"""
## Step 3: Build the RAG System
Combining retrieval with GPT-4 generation
"""
# %%
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Custom prompt to ensure accurate, professional responses
prompt_template = """You are an expert insurance customer service assistant.
Answer the question based only on the following context:

{context}

Question: {question}

Answer professionally in 2-3 sentences. If you don't know, say "I don't have that information but our support team can help at 1-800-INSURANCE"."""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)
# %% [markdown]

/tmp/ipython-input-2777139611.py:25: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


NameError: name 'vector_db' is not defined

In [ ]:
"""
## Step 4: Create the Chatbot Interface
We'll use Streamlit for the web interface
"""
# %%
%%writefile app.py
import streamlit as st
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import os

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []

# UI Setup
st.set_page_config(page_title="Insurance Assistant", page_icon="🏦")
st.title("🏦 Acme Insurance Assistant")

# Sidebar for API key input
with st.sidebar:
    st.header("Configuration")
    api_key = st.text_input("OpenAI API Key", type="password")
    os.environ['OPENAI_API_KEY'] = api_key

# Load vector database
@st.cache_resource
def load_vector_db():
    embeddings = OpenAIEmbeddings()
    return FAISS.load_local("insurance_vector_db", embeddings)

try:
    vector_db = load_vector_db()

    # Initialize QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
        chain_type="stuff",
        retriever=vector_db.as_retriever(),
        return_source_documents=True
    )

    # Display chat history
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # User input
    if prompt := st.chat_input("Ask about your policy or claim:"):
        st.session_state.messages.append({"role": "user", "content": prompt})

        with st.chat_message("user"):
            st.markdown(prompt)

        with st.chat_message("assistant"):
            with st.spinner("Researching your question..."):
                result = qa_chain({"query": prompt})
                response = result["result"]

                st.markdown(response)
                with st.expander("See sources"):
                    for doc in result['source_documents']:
                        st.caption(f"Source: {doc.metadata['source']}")
                        st.text(doc.page_content[:200] + "...")

            st.session_state.messages.append({"role": "assistant", "content": response})

except Exception as e:
    st.error(f"Error loading the chatbot: {str(e)}")
    st.info("Please make sure you've entered a valid OpenAI API key in the sidebar")
# %% [markdown]
"""
## Step 5: Launch the Chatbot
Run this cell to start the Streamlit interface
"""
# %%
set_up_colab()
!streamlit run app.py --server.port 8501 &>/dev/null&
# %% [markdown]
